# Overview

- this is a baseline model to determine immediate performance
- will use gradient boost modelling using package XGboost
- no data cleaning feature engineering to be used

In [2]:
# General Imports

import numpy as np
import pandas as pd
import pickle

import os, gc, sys, warnings, random, psycopg2

In [3]:
# Functions

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [4]:
SEED = 42
LOCAL_TEST = False
seed_everything(SEED)
TARGET = 'meter_reading'

In [5]:
# open pickle files

train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')
submission = pd.read_pickle('submission.pkl')

In [6]:
#data check#data check
print("Main:",list(train_df),train_df.info())
print("#" * 20)
print("Main:",list(test_df),test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 23 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         int16         
 1   site_id             int8          
 2   meter               int8          
 3   meter_reading       float64       
 4   timestamp           datetime64[ns]
 5   primary_use         int8          
 6   square_feet         int32         
 7   year_built          int8          
 8   floor_count         int8          
 9   air_temperature     float64       
 10  cloud_coverage      float16       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float16       
 13  sea_level_pressure  float64       
 14  wind_direction      float16       
 15  wind_speed          float64       
 16  DT_month            int8          
 17  DT_week_y           int8          
 18  DT_day_y            int8          
 19  DT_hour             int8          
 20  

In [7]:
# Check memory usage

In [9]:
#prepare training data
import xgboost as xgb

#choose columns to remove
remove_columns = [TARGET,'timestamp']
#get feature columns using list comprehension to remove columns - list(train_df)
feature_columns = [col for col in list(train_df) if col not in remove_columns]

#use np.log1 for distribution
#label is target

tr_data = xgb.DMatrix(train_df[feature_columns],label=np.log1p(train_df[TARGET]))

In [10]:
# Model Parameters


xgb_param = {'max_depth': 2, 
         'eta': 1, 
         'objective': 'reg:squaredlogerror'} 
    
bst = xgb.train(xgb_param,tr_data,10)

In [11]:
tst_data = xgb.DMatrix(test_df[feature_columns])

In [12]:
predictions = list(np.expm1(bst.predict(tst_data)))

In [13]:
predictions

[5191.288,
 5191.288,
 5191.288,
 5191.288,
 13806.044,
 28950.807,
 5191.288,
 13806.044,
 28950.807,
 2.6481256,
 19135.25,
 13806.044,
 0.7397828,
 2858.8018,
 13806.044,
 13806.044,
 28950.807,
 2.6481256,
 19135.25,
 1.4112984,
 19135.25,
 1.4112984,
 17392.371,
 8880.666,
 28950.807,
 13806.044,
 28950.807,
 5191.288,
 5191.288,
 2858.8018,
 28950.807,
 28950.807,
 13806.044,
 19135.25,
 17392.371,
 1.1916848,
 28950.807,
 2.6481256,
 19135.25,
 1.4112984,
 19135.25,
 1.4112984,
 13806.044,
 13806.044,
 13806.044,
 13806.044,
 17392.371,
 17392.371,
 8880.666,
 19135.25,
 13806.044,
 19135.25,
 2858.8018,
 19135.25,
 1.4112984,
 -1.0,
 8880.666,
 8880.666,
 13806.044,
 8880.666,
 13806.044,
 5191.288,
 -0.3457362,
 2858.8018,
 28950.807,
 19135.25,
 5191.288,
 13806.044,
 0.7397828,
 19135.25,
 19135.25,
 19135.25,
 19135.25,
 19135.25,
 1.4112984,
 13806.044,
 13806.044,
 13806.044,
 17392.371,
 17392.371,
 13806.044,
 13806.044,
 2858.8018,
 2858.8018,
 2858.8018,
 2858.8018,
 

In [44]:
https://www.kaggle.com/volker48/xgboost-custom-rmsle-eval-function

SyntaxError: invalid syntax (<ipython-input-44-836ec6fbd610>, line 1)

In [ ]:
xgb.cv(xgb_param,tr_data,num_boost_round=10,nfold=5,verbose_eval=True)